<font size=6  color=#045fba> <b>[LINMA1702] - Modèles et méthodes d'optimisation I</b> <br><br> 
<b></b>Utilisation optimale d'une pompe à chaleur
domestique (première partie)</font> <br><br><br>

<font size=6  color=#045fba> <strong>Groupe 13 </strong></font> <br><br><br>


<font size=5  color=#045fba>
Simon Cornelis (<i>noma : 55101700</i>)<br>
Nicolas Jeanmenne (<i>noma : 48741900</i>)<br>
Corentin Libert (<i>noma : 53511700</i>)<br>
Aymeric Wibo (<i>noma : 74822100 </i>)<br>

<br><br>
</font>

# Tâche 1 : minimisation du coût total de l'électricité consommée par la pompe à chaleur

## Consignes
<br>
<font size=3>
<div class="alert alert-info">
On souhaite dans un premier temps que la température du bâtiment reste comprise dans une certaine plage admissible de températures, et on cherche à minimiser le coût total de l'électricité consommée par la pompe à chaleur. Formulez ce problème comme un problème d'optimisation linéaire, puis résolvez le. <br>
    
Pour des raisons de temps de calcul, votre modèle considérera uniquement une période de 7 jours consécutifs. Il fera l'hypothèse que la température initiale au début de la période est égale à la valeur central de la plage admissible, et fera en sorte que la température finale à la fin de la période revienne à la même valeur. Votre code prendra donc en entrée un paramètre indiquant le numéro de l'intervalle de temps qui début la période, qui s'étendra sur $ 7 \times 24 \times 4 = 672$ intervalles de temps
</div>
</font>

<div class="alert alert-warning">
A mentionner :
    
    
- coût minimal + graphique de l'évolution des températures + graphique représentant
l'utilisation de la pompe à chaleur (en distinguant le fonctionnement normal du
fonctionnement _reverse_) + temps de calcul + bref commentaire (maximum 4 lignes
    
    
- pour deux périodes distinctes (placer les résultats côté à côté) : à gauche une période
pré-déterminée (cf. fichier de données), et à droite une seconde période que vous
choisirez en fonction de son intérêt
</div>

## Solution

### Variables

Nous avons différencié le mode **normal** du mode **reverse** en créant deux vecteurs $\mathbf{X_N} \in \mathbb{R}^{672}$ et $\mathbf{X_R} \in \mathbb{R}^{672}$, représentant respectivement la consommation en kWh de la pompe à chaleur pour un intervalle en mode **normal** et en mode **reverse**. 

Nous obtenons donc un total de $ 2 * 672 \text{ (c.f. consignes)} = 1344$ variables.

### Fonction objectif

Nous souhaitons minimiser le coût total de l'électricité consommée par la pompe à chaleur. Sous notation mathématique : 

$$ min \;\; (p*X_N + p*X_R) $$
Où: 
 - $p$ est le vecteurs des prix en fonction de la plage horaire des intervalles dans $X_N$ et $X_R$ (tarif bi-horaire: 0,18  $\$$/kWh ou 26 $\$$/kWh).
 - $X_N$ et $X_R$ sont définis comme dans la section [Variables](#Variables).
 
### Contraintes

Nous devons optimiser la fonction objectif sous les contraintes suivantes : 
- La température intérieur (à la fin de chaque intervalle) doit se trouver dans la plage admissible $[T_{min}, T_{max}]$: 

$$ T_{min} \leq T_{int} \leq T_{max}$$
- Nous devons ajouter une contrainte vérifiant, pour un intervalle de temps donné, que la consommation électrique maximal de la pompe à chaleur ne dépasse pas **1kWh**, c'est-à-dire : $$ 0 \leq X_{N, i} + X_{R,i} \leq 1 \text{ kWh} $$


In [82]:
import numpy as np

def get_end_temp(t_start, t_ext, eta=0.99):
    return (1 - eta) * (t_ext - t_start) + t_start

def get_cop(t_ext=None, mode='reverse', size=672):
    if(mode == 'reverse'):
        return np.array([3.2]*size)
    else:
        res = []
        for i in range(size):
            cop = 3 + 10 * np.abs(np.tanh(t_ext[i]/100)) * np.tanh(t_ext[i]/100)
            res.append(cop)
        return res
    
def get_temp_data(data, index, size=672):
    return data[index:index+size]

def compute_end_interval_temp(t_start, t_ext_data, eta=0.99):
    res = []
    for i in range(len(t_ext_data)):
        t_end = get_end_temp(t_start, t_ext_data[i], eta)
        res.append(t_end)
        t_start = 19 if t_end < 19 else 21 if t_end > 21 else t_end
    return res

In [84]:
import numpy as np

data_vector = np.load('data/Temperatures-Montreal.npy')

In [85]:
import numpy as np
import cvxpy as cp
import pandas as pd

# Problem data
n = 672
p = np.array([0.18]*672) # prices
Tmin = 19
Tmax = 21
T_ext = get_temp_data(data_vector, 13050)
T = compute_end_interval_temp(19, T_ext)
cop_n = get_cop(T_ext, 'normal', len(T_ext))
cop_r = get_cop('reverse')


# Variables
Xn = cp.Variable(n)
Xr = cp.Variable(n)
# Objectif
objective = cp.Minimize(p @ Xn + p @ Xr)
# Constraints
constraints = [
    0 <= Xn,
    0 <= Xr,
    Xn + Xr <= 1,
    Tmin <= T + cp.multiply(cop_n, Xn) - cp.multiply(cop_r, Xr),
    T + cp.multiply(cop_n, Xn) - cp.multiply(cop_r, Xr) <= Tmax
]
# Problem
prob = cp.Problem(objective, constraints)


# Solve
result = prob.solve()
print(np.around(result, 6))

# df = pd.DataFrame({"T_ext": T_ext, "T": T })
# display(df.head(20))
# print(Xn.value)

2.739608


In [34]:
import numpy as np
import cvxpy as cp

def get_end_interval_temperature(t_start, t_ext, eta=0.99):
    return (1 - eta) * (t_ext - t_start) + t_start

def get_cop(t_ext=None, mode='reverse', size=672):
    if(mode == 'reverse'):
        return np.array([3.2]*size)
    else:
        cop = 3 + 10 * np.abs(np.tanh(t_ext/100)) * np.tanh(t_ext/100)
        return cop
    


# Tâche 2 : minimisation de l'inconfort total

## Consignes
<br>
<font size=3>
<div class="alert alert-info">
On souhaite réduire le coût d'utilisation de la pompe à chaleur, et on va fixer le budget maximal à une certaine proportion du coût minimal identifié lors de la première tâche. Pour diminuer les coût, on va permettre aux températures de sortir de la plage admissible définie plus haut. On va cependant alors comptabiliser la quantité d'inconfort
éventuellement subi durant chaque intervalle de temps, qui sera proportionnel au dépassement de la température maximale admissible, ou au dépassement par le bas de la température minimale admissible. On cherche alors à <b>minimiser l'inconfort total</b> (somme des inconforts sur toute la période considérée) <b>tout en respectant la contrainte de budget</b>. Formulez ce problème comme un problème d'optimisation linéaire, puis résolvez le.
</div>
</font>

<div class="alert alert-warning">
A mentionner :
    
    
- inconfort minimal + même graphiques que pour tâche 1 + temps de calcul + bref
commentaire (maximum 4 lignes)
    
    
- à nouveau pour les deux périodes mentionnées lors de la tâche 1
</div>

# Tâche 3 : relation entre budget et inconfort

## Consignes
<br>
<font size=3>
<div class="alert alert-info">
On voudrait à présent mieux comprendre le compromis qui existe entre le budget
alloué et l'inconfort total qui en résulte. Proposez un <b>graphique représentant au mieux
cette relation entre budget et inconfort</b>, où on fera varier le budget entre entre zéro et le
coût minimal identifié lors de la tâche 1 (ce budget sera indiqué en pourcentage, de 0 à
100%). Ceci nécessitera la résolution de plusieurs problèmes, et il sera judicieux d'utiliser la
    fonctionnalité <i>warm start</i> du solver pour accélérer les calculs.
</div>
</font>

<div class="alert alert-warning">
A mentionner :
    
    
- graphique demandé + temps de calcul (total et moyenne par problème) + bref
commentaire (maximum 4 lignes)
    
    
- à nouveau pour les deux périodes mentionnées lors des tâches 1 et 2
</div>